In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [23]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [24]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.461049884557724
epoch:  1, loss: 0.24958816170692444
epoch:  2, loss: 0.14683139324188232
epoch:  3, loss: 0.04643009230494499
epoch:  4, loss: 0.04215195029973984
epoch:  5, loss: 0.04018186405301094
epoch:  6, loss: 0.018917107954621315
epoch:  7, loss: 0.018094776198267937
epoch:  8, loss: 0.014612436294555664
epoch:  9, loss: 0.010159029625356197
epoch:  10, loss: 0.008321722969412804
epoch:  11, loss: 0.007406591437757015
epoch:  12, loss: 0.006780920084565878
epoch:  13, loss: 0.006251584272831678
epoch:  14, loss: 0.005870012100785971
epoch:  15, loss: 0.005538708996027708
epoch:  16, loss: 0.005296646151691675
epoch:  17, loss: 0.0050726053304970264
epoch:  18, loss: 0.004946200177073479
epoch:  19, loss: 0.004735412076115608
epoch:  20, loss: 0.0045388732105493546
epoch:  21, loss: 0.004351796582341194
epoch:  22, loss: 0.004202843643724918
epoch:  23, loss: 0.004041582345962524
epoch:  24, loss: 0.003941328264772892
epoch:  25, loss: 0.0037919837050139904
e

In [25]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9575366906204651
Test metrics:  R2 = 0.8785905719300825
